In [3]:
# -*- coding: utf-8 -*-
import os
import re
import glob
import fnmatch
import zipfile
import subprocess
import hashlib
import json
from datetime import datetime, timezone
import xml.etree.ElementTree as ET
from textwrap import dedent
from enum import Enum
import shutil
import numpy as np
import rasterio

In [11]:

# CONFIG
# =========================

STUDY_AREA_NAME = "ARAGON"   # e.g. "Monsanto" for PT case
PLATFORM_CODE   = "ALOS2"    # João: ALOS2, S1GRD, S1SLC, S2
TARGET_CRS      = "EPSG:25830"  # Aragon → UTM 30N

gpt_path   = r"C:\Program Files\esa-snap\bin\gpt.exe"
xml_dir    = r"M:\Project BLS\ALOS 2 PALSAR 2\Pipeline\XML"
output_dir = r"M:\Project BLS\ALOS 2 PALSAR 2\Aragon\Outputs"
export_dir = r"M:\Project BLS\ALOS 2 PALSAR 2\Aragon\Exports"

class PublishMode(str, Enum):
    DEV = "dev"          # keep everything (dims, per-group exports, stamps)
    RELEASE = "release"  # keep only final deliverables

PUBLISH_MODE = PublishMode.RELEASE

KEEP_DIM = (PUBLISH_MODE == PublishMode.DEV)

# "extras" = per-group exports (Exports/<Group>/<tile_id>/...) + Combined stacks if you made them
DELETE_EXTRAS = (PUBLISH_MODE == PublishMode.RELEASE)

# dims deletion only matters if you used tile_out_dir (Outputs/<tile_id>/...)
DELETE_DIMS_AFTER_EXPORT = (PUBLISH_MODE == PublishMode.RELEASE)

# stamps are only useful for DEV
DELETE_DONE_STAMPS = (PUBLISH_MODE == PublishMode.RELEASE)

# Force per-band export for these groups (to avoid huge multibands before reprojection)
FORCE_PER_BAND_GROUPS = {
    "Raw_Backscatter",
    "dB_Backscatter",
    "Speckle_Filtered",
    "Polarization_metrics",
    "GLCM",
    "LIA",
}

# Cache/skip toggle
ENABLE_SKIP_CACHE = True
# If a .done stamp is missing, skip based on timestamps and create it
ALLOW_TIMESTAMP_FALLBACK = True

# ---- Delivery flags ----
DO_COMBINED_STACKS = False  # don't create Combined/*.tif by default

# ---- NoData convention (João) ----
NODATA_VALUE = -9999  # used for all exported GeoTIFFs (incl. GLCM)

# =========================
# Input discovery (dir or .zip)
# =========================


def _find_members_by_suffix(zf: zipfile.ZipFile, suffix: str):
    for name in zf.namelist():
        if name.endswith(suffix):
            return name
    return None

def _to_vsizip(zip_path, inner_path):
    """Return GDAL/SNAP-compatible VSIZIP path."""
    zip_path = zip_path.replace("\\", "/")
    inner_path = inner_path.lstrip("/").replace("\\", "/")
    return f"/vsizip/{zip_path}/{inner_path}"

def _verify_from_zip(zip_path):
    """
    For zipped tiles:
      - Use rasterio + /vsizip/ to *inspect* shapes.
      - Materialize the HH, HV, incidence TIFs to a temp folder on disk.
      - Return *physical* file paths for SNAP.
    """
    expected_suffixes = {
        "HH": "_sl_HH_F02DAR.tif",
        "HV": "_sl_HV_F02DAR.tif",
        "incidence": "_linci_F02DAR.tif"
    }
    file_paths = {}

    # temp folder next to the zip, under _unzipped/<tile_name>/
    zip_dir = os.path.dirname(zip_path)
    tile_name = os.path.splitext(os.path.basename(zip_path))[0]
    temp_root = os.path.join(zip_dir, "_unzipped", tile_name)
    os.makedirs(temp_root, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zf:
        for key, suffix in expected_suffixes.items():
            member = _find_members_by_suffix(zf, suffix)
            if not member:
                raise FileNotFoundError(f"Missing file with suffix '{suffix}' inside zip: {zip_path}")

            # 1) Use /vsizip/ just to read shape (GDAL/rasterio only)
            vsi = _to_vsizip(zip_path, member)
            with rasterio.open(vsi) as src:
                print(f"✅ Loaded {key} from zip: shape {src.shape}")

            # 2) Materialize the TIF for SNAP
            out_name = os.path.basename(member)
            out_path = os.path.join(temp_root, out_name)

            if not os.path.exists(out_path):
                # extract just this member
                with zf.open(member) as src_f, open(out_path, "wb") as dst_f:
                    shutil.copyfileobj(src_f, dst_f)

            # SNAP will use this on-disk TIF
            file_paths[key] = out_path

    return file_paths

def _verify_from_folder(tile_path):
    expected_suffixes = {
        "HH": "_sl_HH_F02DAR.tif",
        "HV": "_sl_HV_F02DAR.tif",
        "incidence": "_linci_F02DAR.tif"
    }
    file_paths = {}
    for key, suffix in expected_suffixes.items():
        matches = glob.glob(os.path.join(tile_path, f"*{suffix}"))
        if not matches:
            raise FileNotFoundError(f"Missing file with suffix: {suffix} in {tile_path}")
        file_paths[key] = matches[0]
        with rasterio.open(file_paths[key]) as src:
            print(f"✅ Loaded {key}: shape {src.shape}")
    return file_paths

def verify_tile_inputs(tile_path):
    if os.path.isfile(tile_path) and tile_path.lower().endswith(".zip"):
        return _verify_from_zip(tile_path)
    elif os.path.isdir(tile_path):
        return _verify_from_folder(tile_path)
    else:
        raise FileNotFoundError(f"tile_path must be a folder with TIFs or a .zip archive. Got: {tile_path}")

def parse_year_and_tile_code(name: str):
    """
    Handles both:
      - N41E000_21_MOS_F02DAR.zip  -> year 2021
      - N41E000_2021              -> year 2021

    Returns:
      year_token = "20210000"
      tile_code  = "N41E000"
    """
    base = os.path.splitext(name)[0]

    # 1) tile_code like N41E000 or N40W001
    m_tile = re.search(r"N\d{2}[EW]\d{3}", base)
    if not m_tile:
        raise ValueError(f"❌ Could not determine tile code from '{name}'")
    tile_code = m_tile.group(0)

    # 2A) first try 2-digit year between underscores: _21_
    m_yy = re.search(r"_(\d{2})_", base)

    if m_yy:
        yy = int(m_yy.group(1))
        year = 2000 + yy   # 21 -> 2021

    else:
        # 2B) otherwise try full 4-digit year after underscore: _2021 or _2021 at end
        m_yyyy = re.search(r"_(20\d{2})(?:_|$)", base)
        if not m_yyyy:
            raise ValueError(f"❌ Could not determine year from '{name}'")
        year = int(m_yyyy.group(1))

    year_token = f"{year}0000"  # João's pattern: YYYY0000
    return year_token, tile_code

def derive_variable_from_stem(stem: str) -> str:
    """
    Convert SNAP-style names to João-style Variable tokens:
      Beta0_HH          -> BETAHH
      Gamma0_HV_dB      -> GAMMAHVDB
      Gamma0_HH_GLCMVariance -> GAMMAHHVARIANCE
      Beta0_HV_ASM      -> BETAHVASM
      Gamma0_ND         -> GAMMAND
      LIA_degrees       -> LIA
    """
    parts = stem.split("_")

    # LIA special case
    if parts[0].upper().startswith("LIA"):
        return "LIA"

    base = parts[0]  # Gamma0, Beta0, Sigma0, etc.
    base_upper = base.upper().replace("0", "")  # Gamma0 -> GAMMA, Beta0 -> BETA

    # If second part is polarization or ND/ratio
    pol = ""
    extra_tokens = []

    if len(parts) >= 2:
        second = parts[1]
        # pol like HH, HV, VH, VV
        if second.upper() in ("HH", "HV", "VH", "VV"):
            pol = second.upper()
            extra_tokens = parts[2:]
        else:
            # e.g. ND / ratio / degrees etc.
            pol = second.upper()
            extra_tokens = parts[2:]

    # Build core (BETA + HH → BETAHH, GAMMA + ND → GAMMAND, etc.)
    core = base_upper + pol

    # Process extra tail
    extras = []
    for t in extra_tokens:
        # strip GLCM prefix if exists
        if t.startswith("GLCM"):
            t = t.replace("GLCM", "", 1)
        extras.append(t.upper())

    return core + "".join(extras)

def _iter_singleband_snap_exports(export_dir, tile_id, groups=None):
    if groups is None:
        groups = ["Raw_Backscatter", "dB_Backscatter", "Speckle_Filtered",
                  "Polarization_metrics", "GLCM", "LIA"]

    for g in groups:
        gdir = os.path.join(export_dir, g, tile_id)
        if not os.path.isdir(gdir):
            continue
        for fname in sorted(os.listdir(gdir)):
            if fname.lower().endswith(".tif"):
                yield g, os.path.join(gdir, fname), fname

def gdal_flatten_and_reproject_for_tile(export_dir, tile_id, year_token, tile_code):
    """
    Final deliverables:
      Exports/<tile_id>/<Group>/StudyArea_YYYY0000_Platform_Variable_TileCode.tif
    """
    tile_root = os.path.join(export_dir, tile_id)
    os.makedirs(tile_root, exist_ok=True)

    for group, in_path, fname in _iter_singleband_snap_exports(export_dir, tile_id):
        stem = os.path.splitext(fname)[0]
        variable = derive_variable_from_stem(stem)

        # resampling per group
        resamp = "near" if group in ("GLCM", "Polarization_metrics") else "bilinear"

        # ✅ group-specific folder prevents overwrites
        group_out_dir = os.path.join(tile_root, group)
        os.makedirs(group_out_dir, exist_ok=True)

        out_name = f"{STUDY_AREA_NAME}_{year_token}_{PLATFORM_CODE}_{variable}_{tile_code}.tif"
        out_path = os.path.join(group_out_dir, out_name)

        cmd = [
            "gdalwarp",
            "-t_srs", TARGET_CRS,
            "-r", resamp,
            "-srcnodata", str(NODATA_VALUE),
            "-dstnodata", str(NODATA_VALUE),
            in_path,
            out_path,
        ]
        print("🧰 GDAL:", " ".join(cmd))
        res = subprocess.run(cmd, capture_output=True, text=True)
        if res.returncode != 0:
            print(res.stderr)
            raise RuntimeError(f"gdalwarp failed on {in_path}")
        else:
            print(f"✅ GDAL reprojected → {os.path.join(group, out_name)}")

# =========================
# Skip-cache helpers
# =========================
def _cache_key(xml_path:str, params:dict):
    """Hash of XML content + relevant params (sorted)."""
    try:
        with open(xml_path, "r", encoding="utf-8") as f:
            xml_txt = f.read()
    except Exception:
        xml_txt = ""
    base = json.dumps({"xml": xml_txt, "params": dict(sorted(params.items()))}, sort_keys=True)
    return hashlib.sha256(base.encode("utf-8")).hexdigest()

def _stamp_path(out_path:str):
    return out_path + ".done"

def _inputs_mtime(inputs):
    mt = 0.0
    for p in inputs:
        if p and os.path.exists(p):
            mt = max(mt, os.path.getmtime(p))
    return mt

def _should_skip(out_path:str, key:str, inputs:list):
    """
    Skip if:
      - out_path exists,
      - sidecar .done exists with same key,
      - and all inputs are older than the out file.
    """
    if not ENABLE_SKIP_CACHE:
        return False
    if not os.path.exists(out_path):
        return False
    sp = _stamp_path(out_path)
    if not os.path.exists(sp):
        return False
    try:
        with open(sp, "r", encoding="utf-8") as f:
            meta = json.load(f)
        if meta.get("key") != key:
            return False
    except Exception:
        return False
    out_mtime = os.path.getmtime(out_path)
    in_mtime  = _inputs_mtime(inputs)
    return in_mtime <= out_mtime

def _inputs_newest_mtime(inputs):
    mt = 0.0
    for p in inputs:
        if p and os.path.exists(p):
            mt = max(mt, os.path.getmtime(p))
    return mt

def _out_is_newer_than_inputs(out_path:str, inputs:list) -> bool:
    if not os.path.exists(out_path):
        return False
    out_mtime = os.path.getmtime(out_path)
    in_mtime  = _inputs_newest_mtime(inputs)
    return in_mtime <= out_mtime

def _write_stamp(out_path:str, key:str, extra:dict=None):
    if not ENABLE_SKIP_CACHE:
        return
    meta = {"key": key, "timestamp": datetime.now(timezone.utc).isoformat()}
    if extra:
        meta.update(extra)
    try:
        with open(_stamp_path(out_path), "w", encoding="utf-8") as f:
            json.dump(meta, f)
    except Exception:
        pass

def _dim_and_data(path:str):
    """Return [dim, dim.data] if present, for timestamp checks."""
    items = [path]
    dd = os.path.splitext(path)[0] + ".data"
    if os.path.isdir(dd):
        items.append(dd)
    return items

# =========================
# GPT templates (self-writing)
# =========================
def _sha(s): return hashlib.sha256(s.encode("utf-8")).hexdigest()

GRAPH_TEMPLATES = {
    # --- Initial 3 band-renames (read TIF → rename band_1) ---
    "Gamma0_HH.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Rename_Gamma0_HH">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Gamma0_HH</name><type>float32</type><expression>band_1</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters>
            <file>${file}</file>
          </parameters></node>
        </graph>
    """),
    "Gamma0_HV.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Rename_Gamma0_HV">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Gamma0_HV</name><type>float32</type><expression>band_1</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters>
            <file>${file}</file>
          </parameters></node>
        </graph>
    """),
    "LIA_degrees.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Rename_LIA_degrees">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>LIA_degrees</name><type>float32</type><expression>band_1</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters>
            <file>${file}</file>
          </parameters></node>
        </graph>
    """),

    # --- LIA deg→rad ---
    "LIA_radian.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_LIA_DegRad">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters>
            <file>${input}</file>
          </parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator>
            <sources><source>Read</source></sources>
            <parameters>
              <targetBands>
                <targetBand><name>LIA_degrees</name><type>float32</type><expression>LIA_degrees</expression></targetBand>
                <targetBand><name>LIA_radians</name><type>float32</type><expression>LIA_degrees * PI / 180</expression></targetBand>
              </targetBands>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator>
            <sources><source>BandMaths</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),

    # --- Collocate HH, HV, LIA ---
    "Collocate_Gamma0_LIA.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Collocate_Gamma0_HH_Gamma0_HV_LIA">
          <version>1.0</version>
          <node id="ReadMaster"><operator>Read</operator><parameters><file>${input1}</file></parameters></node>
          <node id="ReadSlave1"><operator>Read</operator><parameters><file>${input2}</file></parameters></node>
          <node id="ReadSlave2"><operator>Read</operator><parameters><file>${input3}</file></parameters></node>
          <node id="Collocate"><operator>Collocate</operator>
            <sources>
              <master>ReadMaster</master>
              <slave1>ReadSlave1</slave1>
              <slave2>ReadSlave2</slave2>
            </sources>
            <parameters>
              <resamplingType>NEAREST_NEIGHBOUR</resamplingType>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator>
            <sources><source>Collocate</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),

    # --- Sigma0 (from uploaded logic; relies on collocation band names) ---
    "Sigma0_HH.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Sigma0_HH">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources>
            <parameters>
              <targetBands>
                <targetBand><name>Sigma0_HH</name><type>float32</type><expression>if (cos(LIA_radians_S1) > 0) then Gamma0_HH_M * cos(LIA_radians_S1) else NaN</expression></targetBand>
              </targetBands>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),
    "Sigma0_HV.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Sigma0_HV">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources>
            <parameters>
              <targetBands>
                <targetBand><name>Sigma0_HV</name><type>float32</type><expression>if (cos(LIA_radians_S1) > 0) then Gamma0_HV_S0 * cos(LIA_radians_S1) else NaN</expression></targetBand>
              </targetBands>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),

    # --- Beta0 (from uploaded logic) ---
    "Beta0_HH.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Beta0_HH">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources>
            <parameters>
              <targetBands>
                <targetBand><name>Beta0_HH</name><type>float32</type><expression>Gamma0_HH_M / tan(LIA_radians_S1)</expression></targetBand>
              </targetBands>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),
    "Beta0_HV.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="BandMaths_Beta0_HV">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources>
            <parameters>
              <targetBands>
                <targetBand><name>Beta0_HV</name><type>float32</type><expression>Gamma0_HV_S0 / tan(LIA_radians_S1)</expression></targetBand>
              </targetBands>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),

    # --- Merge of 6 backscatter bands ---
    "Merge_Backscatter.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Merge_Backscatter">
          <version>1.0</version>
          <node id="Read_BetaHH"><operator>Read</operator><parameters><file>${beta_hh}</file></parameters></node>
          <node id="Read_BetaHV"><operator>Read</operator><parameters><file>${beta_hv}</file></parameters></node>
          <node id="Read_GammaHH"><operator>Read</operator><parameters><file>${gamma_hh}</file></parameters></node>
          <node id="Read_GammaHV"><operator>Read</operator><parameters><file>${gamma_hv}</file></parameters></node>
          <node id="Read_SigmaHH"><operator>Read</operator><parameters><file>${sigma_hh}</file></parameters></node>
          <node id="Read_SigmaHV"><operator>Read</operator><parameters><file>${sigma_hv}</file></parameters></node>
          <node id="BandMerge"><operator>BandMerge</operator>
            <sources>
              <sourceProducts>Read_BetaHH,Read_BetaHV,Read_GammaHH,Read_GammaHV,Read_SigmaHH,Read_SigmaHV</sourceProducts>
            </sources>
            <parameters/>
          </node>
          <node id="Write"><operator>Write</operator>
            <sources><source>BandMerge</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),

    # --- Speckle filter ---
    "Speckle_Filter_AllBands.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Speckle_Filter_AllBands">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="Speckle_Filter"><operator>Speckle-Filter</operator>
            <sources><sourceProduct refid="Read"/></sources>
            <parameters>
              <filter>Lee</filter>
              <filterSizeX>7</filterSizeX>
              <filterSizeY>7</filterSizeY>
              <dampingFactor>2</dampingFactor>
              <estimateENL>true</estimateENL>
              <enl>1.0</enl>
              <numLooksStr>1</numLooksStr>
              <windowSize>7x7</windowSize>
              <targetWindowSizeStr>3x3</targetWindowSizeStr>
              <sigmaStr>0.9</sigmaStr>
              <anSize>50</anSize>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator>
            <sources><source refid="Speckle_Filter"/></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """),

    # --- Rename _Spk suffix ---
    "Backscatter_Coefficients_Spk_Renamed_All.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Rename_All_Spk">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Beta0_HH_Spk</name><type>float32</type><expression>Beta0_HH</expression></targetBand>
              <targetBand><name>Beta0_HV_Spk</name><type>float32</type><expression>Beta0_HV</expression></targetBand>
              <targetBand><name>Gamma0_HH_Spk</name><type>float32</type><expression>Gamma0_HH</expression></targetBand>
              <targetBand><name>Gamma0_HV_Spk</name><type>float32</type><expression>Gamma0_HV</expression></targetBand>
              <targetBand><name>Sigma0_HH_Spk</name><type>float32</type><expression>Sigma0_HH</expression></targetBand>
              <targetBand><name>Sigma0_HV_Spk</name><type>float32</type><expression>Sigma0_HV</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters><file>${file}</file></parameters></node>
        </graph>
    """),

    # --- dB conversion ---
    "Backscatter_Coefficients_Spk_dB_All.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="All_dB_Conversion">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Beta0_HH_dB</name><type>float32</type><expression>10 * log10(Beta0_HH_Spk) - 83.0</expression></targetBand>
              <targetBand><name>Beta0_HV_dB</name><type>float32</type><expression>10 * log10(Beta0_HV_Spk) - 83.0</expression></targetBand>
              <targetBand><name>Gamma0_HH_dB</name><type>float32</type><expression>10 * log10(Gamma0_HH_Spk) - 83.0</expression></targetBand>
              <targetBand><name>Gamma0_HV_dB</name><type>float32</type><expression>10 * log10(Gamma0_HV_Spk) - 83.0</expression></targetBand>
              <targetBand><name>Sigma0_HH_dB</name><type>float32</type><expression>10 * log10(Sigma0_HH_Spk) - 83.0</expression></targetBand>
              <targetBand><name>Sigma0_HV_dB</name><type>float32</type><expression>10 * log10(Sigma0_HV_Spk) - 83.0</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters><file>${file}</file></parameters></node>
        </graph>
    """),

    # --- Polarization metrics ---
    "Gamma0_PolMetrics.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Gamma0PolarizationMetrics">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Gamma0_ratio</name><type>float32</type><expression>Gamma0_HH / Gamma0_HV</expression></targetBand>
              <targetBand><name>Gamma0_ND</name><type>float32</type><expression>(Gamma0_HH - Gamma0_HV) / (Gamma0_HH + Gamma0_HV)</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters><file>${file}</file></parameters></node>
        </graph>
    """),
    "Sigma0_PolMetrics.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Sigma0PolarizationMetrics">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Sigma0_ratio</name><type>float32</type><expression>Sigma0_HH / Sigma0_HV</expression></targetBand>
              <targetBand><name>Sigma0_ND</name><type>float32</type><expression>(Sigma0_HH - Sigma0_HV) / (Sigma0_HH + Sigma0_HV)</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters><file>${file}</file></parameters></node>
        </graph>
    """),
    "Beta0_PolMetrics.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="Beta0PolarizationMetrics">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="BandMaths"><operator>BandMaths</operator><sources><source>Read</source></sources><parameters>
            <targetBands>
              <targetBand><name>Beta0_ratio</name><type>float32</type><expression>Beta0_HH / Beta0_HV</expression></targetBand>
              <targetBand><name>Beta0_ND</name><type>float32</type><expression>(Beta0_HH - Beta0_HV) / (Beta0_HH + Beta0_HV)</expression></targetBand>
            </targetBands>
          </parameters></node>
          <node id="Write"><operator>Write</operator><sources><source>BandMaths</source></sources><parameters><file>${file}</file></parameters></node>
        </graph>
    """),

    # --- GLCM (baseline params) ---
    "GLCM_AllBands.xml": dedent("""\
        <?xml version="1.0" encoding="UTF-8"?>
        <graph id="GLCM_AllBands">
          <version>1.0</version>
          <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
          <node id="GLCM"><operator>GLCM</operator>
            <sources><source>Read</source></sources>
            <parameters>
              <windowSizeStr>5x5</windowSizeStr>
              <quantizerStr>Probabilistic Quantizer</quantizerStr>
              <quantizationLevelsStr>64</quantizationLevelsStr>
              <displacement>1</displacement>
              <noDataValue>-9999.0</noDataValue>
              <outputContrast>true</outputContrast>
              <outputDissimilarity>true</outputDissimilarity>
              <outputEntropy>true</outputEntropy>
              <outputEnergy>true</outputEnergy>
              <outputASM>true</outputASM>
              <outputMean>true</outputMean>
              <outputVariance>true</outputVariance>
              <outputCorrelation>true</outputCorrelation>
              <outputHomogeneity>true</outputHomogeneity>
            </parameters>
          </node>
          <node id="Write"><operator>Write</operator><sources><source>GLCM</source></sources>
            <parameters><file>${file}</file></parameters>
          </node>
        </graph>
    """)
}

def ensure_processing_graphs(xml_dir):
    os.makedirs(xml_dir, exist_ok=True)
    for name, content in GRAPH_TEMPLATES.items():
        path = os.path.join(xml_dir, name)
        want = content.strip()
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as f:
                have = f.read().strip()
            if _sha(have) == _sha(want):
                continue
        with open(path, "w", encoding="utf-8") as f:
            f.write(want)
        print(f"🧩 Installed/updated graph: {name}")

def run_gpt_with_params(gpt_path, xml_path, params):
    cmd = [gpt_path, xml_path] + [f"-P{k}={v}" for k, v in params.items()]
    pretty = " ".join(f'"{c}"' if " " in c else c for c in cmd)
    print("🧰", pretty)

    res = subprocess.run(cmd, capture_output=True, text=True)

    # Show SNAP output if present (useful when caching is off)
    if res.stdout.strip():
        print(res.stdout.strip())

    if res.returncode != 0:
        # Bubble up helpful error info
        err = res.stderr.strip() or "Unknown SNAP error"
        print(err)
        raise RuntimeError(f"Graph failed: {os.path.basename(xml_path)}\n{err}")

    print(f"✅ {os.path.basename(xml_path)}")
    return res  # (optional) lets callers inspect stdout/stderr if needed

# =========================
# Export helpers (pure GPT)
# =========================
def list_dim_band_names(dim_path):
    try:
        tree = ET.parse(dim_path)
        root = tree.getroot()
        names = [el.text for el in root.findall(".//Band/Name") if el is not None and el.text]
        if not names:
            names = [el.text for el in root.findall(".//BAND_NAME") if el is not None and el.text]
        return names
    except Exception as e:
        print(f"⚠️ Could not parse band names from {dim_path}: {e}")
        return []

def ensure_export_xml_templates(xml_dir):
    """Two tiny export graphs: all-bands and single-band."""
    all_bands_xml = os.path.join(xml_dir, "ExportAllBandsToGTiff.xml")
    single_band_xml = os.path.join(xml_dir, "ExportSingleBandToGTiff.xml")

    if not os.path.exists(all_bands_xml):
        with open(all_bands_xml, "w", encoding="utf-8") as f:
            f.write(dedent("""\
                <?xml version="1.0" encoding="UTF-8"?>
                <graph id="ExportAllBandsToGTiff">
                  <version>1.0</version>
                  <node id="Read"><operator>Read</operator><parameters><file>${input}</file></parameters></node>
                  <node id="Write"><operator>Write</operator><sources><source>Read</source></sources>
                    <parameters><file>${out}</file><formatName>GeoTIFF</formatName></parameters>
                  </node>
                </graph>
            """).strip())

    if not os.path.exists(single_band_xml):
        with open(single_band_xml, "w", encoding="utf-8") as f:
            f.write(dedent(f"""\
                <?xml version="1.0" encoding="UTF-8"?>
                <graph id="ExportSingleBandToGTiff">
                  <version>1.0</version>
                  <node id="Read">
                    <operator>Read</operator>
                    <parameters><file>${{input}}</file></parameters>
                  </node>
                  <node id="Subset">
                    <operator>Subset</operator>
                    <sources><source>Read</source></sources>
                    <parameters>
                      <sourceBands>${{bandName}}</sourceBands>
                      <copyMetadata>true</copyMetadata>
                    </parameters>
                  </node>
                  <node id="BandMaths_NoData">
                    <operator>BandMaths</operator>
                    <sources><source>Subset</source></sources>
                    <parameters>
                      <targetBands>
                        <targetBand>
                          <name>${{bandName}}</name>
                          <type>float32</type>
                          <expression>if isNaN(${{bandName}}) then {NODATA_VALUE} else ${{bandName}}</expression>
                          <noDataValue>{NODATA_VALUE}</noDataValue>
                        </targetBand>
                      </targetBands>
                    </parameters>
                  </node>
                  <node id="Write">
                    <operator>Write</operator>
                    <sources><source>BandMaths_NoData</source></sources>
                    <parameters>
                      <file>${{out}}</file>
                      <formatName>GeoTIFF</formatName>
                    </parameters>
                  </node>
                </graph>
            """).strip())
    return all_bands_xml, single_band_xml

def export_all_bands_dim_to_tiff(gpt_path, xml_dir, dim_path, out_tif):
    xml = os.path.join(xml_dir, "ExportAllBandsToGTiff.xml")
    run_gpt_with_params(gpt_path, xml, {"input": dim_path, "out": out_tif})

def export_each_band_dim_to_folder(gpt_path, xml_dir, dim_path, out_dir, prefix=""):
    os.makedirs(out_dir, exist_ok=True)
    band_names = list_dim_band_names(dim_path)
    if not band_names:
        print(f"⚠️ No bands discovered in {dim_path}; skipping per-band export.")
        return
    xml = os.path.join(xml_dir, "ExportSingleBandToGTiff.xml")
    for b in band_names:
        safe = b.replace(" ", "_").replace("/", "_")
        out_tif = os.path.join(out_dir, f"{prefix}{safe}.tif")
        params = {"input": dim_path, "bandName": b, "out": out_tif}
        key = _cache_key(xml, params)
        # inputs for timestamp compare: the .dim and its .data folder (if exists)
        inputs = [dim_path]
        data_dir = os.path.splitext(dim_path)[0] + ".data"
        if os.path.isdir(data_dir): inputs.append(data_dir)
        if _should_skip(out_tif, key, inputs):
            print(f"⏭️  Skip export (cached): {out_tif}")
            continue
        run_gpt_with_params(gpt_path, xml, params)
        _write_stamp(out_tif, key)

def hybrid_publish_exports(*, gpt_path, xml_dir, export_dir, tile_id, tile_out_dir, big_group_threshold=12):
    """
    Writes per-band exports into: export_dir/<Group>/<tile_id>/
    """
    ensure_export_xml_templates(xml_dir)
    os.makedirs(export_dir, exist_ok=True)

    products = [
        ("Raw_Backscatter", os.path.join(tile_out_dir, "Backscatter_Coefficients_B_G_S.dim")),
        ("dB_Backscatter",  os.path.join(tile_out_dir, "Backscatter_Coefficients_Spk_dB_All.dim")),
        ("Speckle_Filtered",os.path.join(tile_out_dir, "Backscatter_Coefficients_Speckle_AllBands.dim")),
        ("Polarization_metrics", os.path.join(tile_out_dir, "Gamma0_PolMetrics.dim")),
        ("Polarization_metrics", os.path.join(tile_out_dir, "Sigma0_PolMetrics.dim")),
        ("Polarization_metrics", os.path.join(tile_out_dir, "Beta0_PolMetrics.dim")),
        ("GLCM",            os.path.join(tile_out_dir, "Backscatter_Coefficients_GLCM_AllBands.dim")),
        ("LIA",             os.path.join(tile_out_dir, "LIA_degrees.dim")),
    ]

    for group, dim_path in products:
        if not os.path.exists(dim_path):
            print(f"⚠️ Missing product: {dim_path}")
            continue

        band_names = list_dim_band_names(dim_path)
        group_dir = os.path.join(export_dir, group, tile_id)  # <-- per-tile folder
        os.makedirs(group_dir, exist_ok=True)

        # 👉 Force per-band export for the chosen groups
        if (group in FORCE_PER_BAND_GROUPS) or (band_names and len(band_names) > big_group_threshold):
            print(f"📦 {group}: per-band export ({len(band_names)} bands) → {group_dir}")
            export_each_band_dim_to_folder(gpt_path, xml_dir, dim_path, group_dir, prefix="")
        else:
            out_tif = os.path.join(group_dir, os.path.splitext(os.path.basename(dim_path))[0] + ".tif")
            params = {"input": dim_path, "out": out_tif}
            xml = os.path.join(xml_dir, "ExportAllBandsToGTiff.xml")
            key = _cache_key(xml, params)
            inputs = [dim_path]
            data_dir = os.path.splitext(dim_path)[0] + ".data"
            if os.path.isdir(data_dir): inputs.append(data_dir)
            if _should_skip(out_tif, key, inputs):
                print(f"⏭️  Skip export (cached): {out_tif}")
            else:
                export_all_bands_dim_to_tiff(gpt_path, xml_dir, dim_path, out_tif)
                _write_stamp(out_tif, key)


# =========================
# Stacking + cleanup + README mode stamp
# =========================
def stack_grouped_tiffs(base_export_dir, tile_id, cleanup_extras=True):
    """
    Stacks from per-tile folders:
      base_export_dir/<Group>/<tile_id>/*.tif
    Writes combined to:
      base_export_dir/Combined/<tile_id>/*.tif
    """
    combined_output = os.path.join(base_export_dir, "Combined", tile_id)
    os.makedirs(combined_output, exist_ok=True)

    group_folders = {
        'Raw_Backscatter': 'Raw_Backscatter',
        'Backscatter_dB': 'dB_Backscatter',
        'Speckle_Filtered': 'Speckle_Filtered',
        'Polarization': 'Polarization_metrics',
        'GLCM': 'GLCM'
    }

    exclude_patterns = {
        'Raw_Backscatter': ['Backscatter_Coefficients_B_G_S.tif'],
        'Backscatter_dB': ['Backscatter_Coefficients_Spk_dB_All.tif'],
        'Speckle_Filtered': ['Backscatter_Coefficients_Speckle_AllBands.tif'],
        'Polarization': ['*_PolMetrics.tif'],
    }

    readme_lines = []

    for group_name, subfolder in group_folders.items():
        input_dir = os.path.join(base_export_dir, subfolder, tile_id)  # <-- per-tile source
        output_path = os.path.join(combined_output, f"{group_name}_AllBands.tif")
        if not os.path.exists(input_dir):
            continue

        tif_files = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(".tif")])
        if not tif_files:
            continue

        patterns = exclude_patterns.get(group_name, [])
        def keep(name): return not any(fnmatch.fnmatch(name, pat) for pat in patterns)
        keepers = [f for f in tif_files if keep(f)]
        extras  = [f for f in tif_files if not keep(f)]

        if cleanup_extras and extras:
            for e in extras:
                try:
                    os.remove(os.path.join(input_dir, e))
                    print(f"🗑️ Deleted extra: {e}")
                except Exception as err:
                    print(f"⚠️ Could not delete {e}: {err}")

        if not keepers:
            continue

        if group_name == 'GLCM' and len(keepers) > 12:
            readme_lines.append(f"\n📦 {group_name} ({len(keepers)} files, per-band export)")
            readme_lines += [f"  File: {name}" for name in keepers]
            print(f"ℹ️ Skipping stacking for large group {group_name} (per-band already).")
            continue

        print(f"\n📦 Stacking group: {group_name} ({len(keepers)} bands)")
        readme_lines.append(f"\n📦 {group_name} ({len(keepers)} bands):")

        with rasterio.open(os.path.join(input_dir, keepers[0])) as ref:
            profile = ref.profile
            profile.update(count=len(keepers), dtype='float32')

        with rasterio.open(output_path, 'w', **profile) as dst:
            for idx, fname in enumerate(keepers):
                band_path = os.path.join(input_dir, fname)
                with rasterio.open(band_path) as src:
                    dst.write(src.read(1), idx + 1)
                readme_lines.append(f"  Band {idx + 1}: {os.path.splitext(fname)[0]}")
        print(f"✅ Saved {group_name}_AllBands.tif")

    # Copy LIA degrees (per-tile)
    lia_src = os.path.join(base_export_dir, "LIA", tile_id, "LIA_degrees.tif")
    lia_dst = os.path.join(combined_output, "LIA_degrees.tif")
    if os.path.exists(lia_src):
        shutil.copy2(lia_src, lia_dst)
        print(f"📄 Copied LIA_degrees.tif to Combined folder")

    # README with mode stamp
    readme_path = os.path.join(combined_output, "README_bandmap.txt")
    with open(readme_path, "w", encoding="utf-8") as f:
        f.write(f"Pipeline export mode: {PUBLISH_MODE.value.upper()}\n")
        f.write("===========================================\n\n")
        f.write("\n".join(readme_lines))
    print(f"\n📝 Saved: {readme_path}")
    print("🎉 All grouped TIFFs stacked successfully.")

def cleanup_dim_products(output_dir, keep_patterns=None):
    keep_patterns = keep_patterns or []
    def keep(name): return any(pat in name for pat in keep_patterns)

    for name in os.listdir(output_dir):
        full = os.path.join(output_dir, name)
        if name.lower().endswith(".dim") and not keep(name):
            try:
                os.remove(full); print(f"🗑️ Deleted DIM: {name}")
            except Exception as e:
                print(f"⚠️ Could not delete {name}: {e}")
        if name.lower().endswith(".data") and not keep(name):
            try:
                shutil.rmtree(full); print(f"🗑️ Deleted DATA folder: {name}")
            except Exception as e:
                print(f"⚠️ Could not delete {name}: {e}")

def cleanup_done_stamps(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith(".done"):
                fpath = os.path.join(dirpath, fname)
                try:
                    os.remove(fpath)
                    print(f"🗑️ Deleted stamp: {fpath}")
                except Exception as e:
                    print(f"⚠️ Could not delete stamp {fpath}: {e}")

# =========================
# Ordered processing chain
# =========================
def _with_data(dim_path: str):
    """Return [dim_path] plus its .data folder if present."""
    lst = [dim_path]
    data_dir = os.path.splitext(dim_path)[0] + ".data"
    if os.path.isdir(data_dir):
        lst.append(data_dir)
    return lst

def run_gpt_cached(gpt_path, xml_dir, xml_name, params, out_path, inputs):
    """
    Cached runner that matches your current calls in run_ordered_gpt_tasks.

    - gpt_path: path to gpt.exe
    - xml_dir: directory containing XML graphs
    - xml_name: the XML file name (e.g. "Sigma0_HH.xml")
    - params: dict of -P parameters
    - out_path: expected output product (.dim)
    - inputs: list of input file paths (e.g. .dim + .data)
    """
    xml_path = os.path.join(xml_dir, xml_name)
    key = _cache_key(xml_path, params)

    # Bootstrap: output exists but no .done yet
    if os.path.exists(out_path) and not os.path.exists(_stamp_path(out_path)):
        _write_stamp(out_path, key, extra={"bootstrap": True})
        print(f"⏭️  Skip (bootstrapped): {xml_name} → {out_path}")
        return

    # Normal cached skip
    if _should_skip(out_path, key, inputs):
        print(f"⏭️  Skip (cached): {xml_name} → {out_path}")
        return

    # Otherwise run
    run_gpt_with_params(gpt_path, xml_path, params)
    _write_stamp(out_path, key)

def canonical_tile_id(entry_name: str) -> tuple[str, str, str]:
    year_token, tile_code = parse_year_and_tile_code(entry_name)
    tile_id = f"{tile_code}_{year_token[:4]}"
    return tile_id, year_token, tile_code


def run_ordered_gpt_tasks(gpt_path, xml_dir, output_dir, file_paths):
    ensure_processing_graphs(xml_dir)

    # Paths we’ll reuse
    p_Gamma0_HH   = os.path.join(output_dir, "Gamma0_HH.dim")
    p_Gamma0_HV   = os.path.join(output_dir, "Gamma0_HV.dim")
    p_LIA_deg     = os.path.join(output_dir, "LIA_degrees.dim")
    p_LIA_rad     = os.path.join(output_dir, "LIA_radians.dim")
    p_Collocated  = os.path.join(output_dir, "Gamma0_HH_HV_LIA.dim")
    p_Sigma0_HH   = os.path.join(output_dir, "Sigma0_HH.dim")
    p_Sigma0_HV   = os.path.join(output_dir, "Sigma0_HV.dim")
    p_Beta0_HH    = os.path.join(output_dir, "Beta0_HH.dim")
    p_Beta0_HV    = os.path.join(output_dir, "Beta0_HV.dim")
    p_MergedBGS   = os.path.join(output_dir, "Backscatter_Coefficients_B_G_S.dim")
    p_SpeckleAll  = os.path.join(output_dir, "Backscatter_Coefficients_Speckle_AllBands.dim")
    p_SpkRenamed  = os.path.join(output_dir, "Backscatter_Coefficients_Spk_Renamed_All.dim")
    p_Spk_dB_All  = os.path.join(output_dir, "Backscatter_Coefficients_Spk_dB_All.dim")
    p_Pol_Gamma   = os.path.join(output_dir, "Gamma0_PolMetrics.dim")
    p_Pol_Sigma   = os.path.join(output_dir, "Sigma0_PolMetrics.dim")
    p_Pol_Beta    = os.path.join(output_dir, "Beta0_PolMetrics.dim")
    p_GLCM_All    = os.path.join(output_dir, "Backscatter_Coefficients_GLCM_AllBands.dim")

    # 1) Rename band_1 → Gamma0 & LIA (from source TIF/VSIZIP)
    run_gpt_cached(gpt_path, xml_dir, "Gamma0_HH.xml",
                   {"input": file_paths["HH"], "file": p_Gamma0_HH},
                   p_Gamma0_HH, [file_paths["HH"]])

    run_gpt_cached(gpt_path, xml_dir, "Gamma0_HV.xml",
                   {"input": file_paths["HV"], "file": p_Gamma0_HV},
                   p_Gamma0_HV, [file_paths["HV"]])

    run_gpt_cached(gpt_path, xml_dir, "LIA_degrees.xml",
                   {"input": file_paths["incidence"], "file": p_LIA_deg},
                   p_LIA_deg, [file_paths["incidence"]])

    # 2) LIA deg → rad
    run_gpt_cached(gpt_path, xml_dir, "LIA_radian.xml",
                   {"input": p_LIA_deg, "file": p_LIA_rad},
                   p_LIA_rad, _with_data(p_LIA_deg))

    # 3) Collocate HH, HV, LIA
    run_gpt_cached(gpt_path, xml_dir, "Collocate_Gamma0_LIA.xml",
                   {"input1": p_Gamma0_HH, "input2": p_Gamma0_HV, "input3": p_LIA_rad, "file": p_Collocated},
                   p_Collocated, _with_data(p_Gamma0_HH) + _with_data(p_Gamma0_HV) + _with_data(p_LIA_rad))

    # 4) Sigma0 / Beta0 from collocated
    run_gpt_cached(gpt_path, xml_dir, "Sigma0_HH.xml",
                   {"input": p_Collocated, "file": p_Sigma0_HH},
                   p_Sigma0_HH, _with_data(p_Collocated))

    run_gpt_cached(gpt_path, xml_dir, "Sigma0_HV.xml",
                   {"input": p_Collocated, "file": p_Sigma0_HV},
                   p_Sigma0_HV, _with_data(p_Collocated))

    run_gpt_cached(gpt_path, xml_dir, "Beta0_HH.xml",
                   {"input": p_Collocated, "file": p_Beta0_HH},
                   p_Beta0_HH, _with_data(p_Collocated))

    run_gpt_cached(gpt_path, xml_dir, "Beta0_HV.xml",
                   {"input": p_Collocated, "file": p_Beta0_HV},
                   p_Beta0_HV, _with_data(p_Collocated))

    # 5) Merge 6 backscatter bands
    run_gpt_cached(gpt_path, xml_dir, "Merge_Backscatter.xml",
                   {"beta_hh": p_Beta0_HH, "beta_hv": p_Beta0_HV,
                    "gamma_hh": p_Gamma0_HH, "gamma_hv": p_Gamma0_HV,
                    "sigma_hh": p_Sigma0_HH, "sigma_hv": p_Sigma0_HV,
                    "file": p_MergedBGS},
                   p_MergedBGS, _with_data(p_Beta0_HH)+_with_data(p_Beta0_HV)+
                                _with_data(p_Gamma0_HH)+_with_data(p_Gamma0_HV)+
                                _with_data(p_Sigma0_HH)+_with_data(p_Sigma0_HV))

    # 6) Speckle → rename → dB
    run_gpt_cached(gpt_path, xml_dir, "Speckle_Filter_AllBands.xml",
                   {"input": p_MergedBGS, "file": p_SpeckleAll},
                   p_SpeckleAll, _with_data(p_MergedBGS))

    run_gpt_cached(gpt_path, xml_dir, "Backscatter_Coefficients_Spk_Renamed_All.xml",
                   {"input": p_SpeckleAll, "file": p_SpkRenamed},
                   p_SpkRenamed, _with_data(p_SpeckleAll))

    run_gpt_cached(gpt_path, xml_dir, "Backscatter_Coefficients_Spk_dB_All.xml",
                   {"input": p_SpkRenamed, "file": p_Spk_dB_All},
                   p_Spk_dB_All, _with_data(p_SpkRenamed))

    # 7) Pol metrics (from speckled)
    run_gpt_cached(gpt_path, xml_dir, "Gamma0_PolMetrics.xml",
                   {"input": p_SpeckleAll, "file": p_Pol_Gamma},
                   p_Pol_Gamma, _with_data(p_SpeckleAll))

    run_gpt_cached(gpt_path, xml_dir, "Sigma0_PolMetrics.xml",
                   {"input": p_SpeckleAll, "file": p_Pol_Sigma},
                   p_Pol_Sigma, _with_data(p_SpeckleAll))

    run_gpt_cached(gpt_path, xml_dir, "Beta0_PolMetrics.xml",
                   {"input": p_SpeckleAll, "file": p_Pol_Beta},
                   p_Pol_Beta, _with_data(p_SpeckleAll))

    # 8) GLCM (from speckled)
    run_gpt_cached(gpt_path, xml_dir, "GLCM_AllBands.xml",
                   {"input": p_SpeckleAll, "file": p_GLCM_All},
                   p_GLCM_All, _with_data(p_SpeckleAll))


def process_multiple_tiles(
    data_root,
    gpt_path,
    xml_dir,
    output_dir,
    export_dir,
    max_tiles=1
):
    dir_tiles = [f for f in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, f))]
    zip_tiles = [f for f in os.listdir(data_root) if os.path.isfile(os.path.join(data_root, f)) and f.lower().endswith(".zip")]
    tile_entries = sorted(dir_tiles + zip_tiles)[:max_tiles]
    print(f"🔁 Found {len(tile_entries)} tile entries (dirs+zips). Processing {len(tile_entries)}...\n")

    ensure_processing_graphs(xml_dir)

    for entry in tile_entries:
        tile_path = os.path.join(data_root, entry)
        print(f"\n\n==============================")
        print(f"🚀 Processing tile entry: {entry}")
        print(f"==============================")

        try:
            # 1) inputs (dir or zip)
            file_paths = verify_tile_inputs(tile_path)

            # 2) canonical tile identity (single source of truth)
            tile_id, year_token, tile_code = canonical_tile_id(entry)
            tile_out_dir = os.path.join(output_dir, tile_id)
            os.makedirs(tile_out_dir, exist_ok=True)

            # 3) processing chain (writes into Outputs/<tile_id>/)
            run_ordered_gpt_tasks(gpt_path, xml_dir, tile_out_dir, file_paths)

            # 4) per-band exports into Exports/<Group>/<tile_id>/
            hybrid_publish_exports(
                gpt_path=gpt_path,
                xml_dir=xml_dir,
                tile_out_dir=tile_out_dir,
                export_dir=export_dir,
                tile_id=tile_id,
                big_group_threshold=12
            )

            # 5) optional Combined stacks
            if DO_COMBINED_STACKS:
                stack_grouped_tiffs(export_dir, tile_id, cleanup_extras=DELETE_EXTRAS)

            # 6) optional cleanup of dims for RELEASE
            if DELETE_DIMS_AFTER_EXPORT and not KEEP_DIM:
                cleanup_dim_products(tile_out_dir, keep_patterns=["LIA"])

            # 7) Final GDAL reprojection + João naming into Exports/<tile_id>/
            print(f"\n🌍 GDAL flatten/reproject for tile: {tile_id}")
            gdal_flatten_and_reproject_for_tile(export_dir, tile_id, year_token, tile_code)

            # 8) Optional cleanup for RELEASE
            if PUBLISH_MODE == PublishMode.RELEASE:
                # If you want: delete stamps only under this tile’s export subtree
                # (better than nuking all stamps globally mid-run)
                cleanup_done_stamps(os.path.join(export_dir, tile_id))

            print(f"\n✅ COMPLETED TILE: {tile_id}")

        except Exception as e:
            print(f"\n❌ FAILED TILE: {entry}")
            print(f"   Reason: {str(e)}")



if __name__ == "__main__":
    process_multiple_tiles(
        data_root=r"M:\Project BLS\ALOS 2 PALSAR 2\Data\Aragon",
        gpt_path=gpt_path,
        xml_dir=xml_dir,
        output_dir=output_dir,
        export_dir=export_dir,
        max_tiles=1
    )


🔁 Found 1 tile entries (dirs+zips). Processing 1...



🚀 Processing tile entry: N43E000_21_MOS_F02DAR.zip
✅ Loaded HH from zip: shape (4500, 4500)
✅ Loaded HV from zip: shape (4500, 4500)
✅ Loaded incidence from zip: shape (4500, 4500)
🧰 "C:\Program Files\esa-snap\bin\gpt.exe" "M:\Project BLS\ALOS 2 PALSAR 2\Pipeline\XML\Gamma0_HH.xml" "-Pinput=M:\Project BLS\ALOS 2 PALSAR 2\Data\Aragon\_unzipped\N43E000_21_MOS_F02DAR\N43E000_21_sl_HH_F02DAR.tif" "-Pfile=M:\Project BLS\ALOS 2 PALSAR 2\Aragon\Outputs\N43E000_2021\Gamma0_HH.dim"
Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
✅ Gamma0_HH.xml
🧰 "C:\Program Files\esa-snap\bin\gpt.exe" "M:\Project BLS\ALOS 2 PALSAR 2\Pipeline\XML\Gamma0_HV.xml" "-Pinput=M:\Project BLS\ALOS 2 PALSAR 2\Data\Aragon\_unzipped\N43E000_21_MOS_F02DAR\N43E000_21_sl_HV_F02DAR.tif" "-Pfile=M:\Project BLS\ALOS 2 PALSAR 2\Aragon\Outputs\N43E000_2021\Gamma0_HV.dim"
Executing processing graph
....10%....20%....30%....40%..